<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/sixth_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Six Test - Simulate Inference

Inference with the subtask code implementation, the text is now the most reasonable but it has some sense. The only issue is that the COMPLEMENTARY DATA is on blank. But the task is filling it nice. So there exist probability that another probability that we are calling the wrong COMPLEMENTARY DATA.

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Dependencies Installation
# Not explicitly because the repo is private.


In [ ]:
DS_ID = "NONHUMAN-RESEARCH/TEST_RECORD_ANNOTATIONS"
PRETRAINED_PATH = "lerobot/pi05_base"

In [ ]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset

dataset = LeRobotDataset(DS_ID)

In [ ]:
from xhuman.policies.pi05.configuration_pi05 import  PI05Config
device = "cuda"
policy_config = PI05Config(
        pretrained_path=PRETRAINED_PATH,  # Set pretrained path so factory loads weights
        device=device,
    )

In [ ]:
from xhuman.policies.factory import make_xhuman_policy

policy = make_xhuman_policy(
        cfg=policy_config,
        ds_meta=dataset.meta,
    )

In [ ]:
from xhuman.preprocessors.factory import make_xhuman_pre_post_processors

preprocessor, _ = make_xhuman_pre_post_processors(
        policy_cfg=policy_config,
        pretrained_path=policy_config.pretrained_path,
    )


In [ ]:
def change_task(frame):
  frame["task"] = "Put the fruits in the basket"
  return frame

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get the first image tensor from the list and move it to CPU
image_tensor = img[1].squeeze(0).cpu().detach()

# Permute dimensions from (C, H, W) to (H, W, C) for matplotlib
image_np = image_tensor.permute(1, 2, 0).numpy()

# Normalize pixel values from [-3, 3] to [0, 1] for display
image_np = (image_np + 1) / 2

# Clip values to ensure they are within [0, 1] after normalization
image_np = np.clip(image_np, 0, 1)

# Plot the image
plt.imshow(image_np)
plt.title("Processed Image")
plt.axis('off') # Hide axes ticks and labels
plt.show()

In [ ]:
img, img_mask = policy._preprocess_images(batch_subtask)

In [ ]:
tokens , mask = batch_subtask["observation.language.tokens"], batch_subtask["observation.language.attention_mask"]
# img, batch_subtask["observation.images.top"]

In [ ]:
policy._detokenize_subtask(tokens)

In [ ]:
mask_ai = img_mask[0]
mask_ai

In [ ]:
import torch

In [ ]:
mask_2 = [fal, mask_ai, fal]

In [ ]:
for i in range(1000):
    frame = change_task(dataset[i])
    time_index = frame["frame_index"]

    # Add the cached subtask to the frame BEFORE preprocessing
    if policy.cached_subtask is not None:
        frame["subtask"] = policy.cached_subtask

    if time_index % policy_config.subtask_prediction_frequency == 0:
        print("Time Index", time_index.item())
        batch_subtask = preprocessor.subtask(frame)
        print("Batch Task", batch_subtask["task"][0])
        policy.update_subtask(batch_subtask)
        print("Policy cached_subtask:", policy.cached_subtask)

        # Update frame with new subtask for next iteration
        frame["subtask"] = policy.cached_subtask

    batch = preprocessor(frame)


    if i % 50 == 0:
        print("\n" + "="*80)
        print(f"Step {i} | Time Index: {time_index}")
        print("-"*80)
        print(f"Batch Task (prompt): {batch['task'][0]}")
        print("="*80 + "\n")

Time Index 0
Batch Task Task: put the fruits in the basket. Subtask: 
Policy cached_subtask: left뀜⡯

Step 0 | Time Index: 0
Batch Task (prompt): Task: left뀜⡯. State: -1 0 255 114 77 112 -1 164 -1 255 175 53 20 2 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 


Step 50 | Time Index: 50
Batch Task (prompt): Task: left뀜⡯. State: 7 0 255 101 57 167 -1 212 0 255 164 71 47 0 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 

Time Index 100
Batch Task Task: put the fruits in the basket. Subtask: 
Policy cached_subtask: , the⸂

Step 100 | Time Index: 100
Batch Task (prompt): Task: , the⸂. State: 7 0 255 102 60 160 -1 211 0 255 164 71 47 0 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 


Step 150 | Time Index: 150
Batch Task (prompt): Task: , the⸂. State: 73 42 184 124 151 167 42 161 75 168 163 172 5 66 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 

Time Index 200
Batch Task Task: put the fruits in the basket. Subtask: 
Policy cached_subtask: left舠

Step 200 | Time Index: 200
Batch Task (prompt): Task: left舠. State: 52 50 188 140 128 148 48 60 159 147 255 223 118 255 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 


Step 250 | Time Index: 250
Batch Task (prompt): Task: left舠. State: 8 52 183 160 139 134 45 24 210 157 255 157 169 255 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 

Time Index 300
Batch Task Task: put the fruits in the basket. Subtask: 
Policy cached_subtask: left

Step 300 | Time Index: 300
Batch Task (prompt): Task: left. State: 13 45 193 163 131 125 39 48 198 113 255 199 170 198 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 
